EXERCISE 2: In this exercise I redo the Monte Carlo Simulation of Berry(1994). We have 500 simulated duopoly markets. The utility of each consumer in each market is given by 
$u_{ij} = \beta_0 + \beta_xx_j + \sigma_d\xi_j - \alpha p_j + \epsilon_{ij}$
with $\epsilon_{ij}$ follow T1EV. The utility of outside good is given by $u_{i0} = \epsilon_{i0}$ where $\epsilon_{i0}$ follow the same distribution. Marginal cost is constraint to be positive and is given by 
$c_j = e^{\gamma_0 + \gamma_xx_j+\sigma_c\xi_j + \sigma\xi_j + \gamma_ww_j + \sigma_\omega\omega_j }$

The firm maximizes the profit function $\pi_j = p_jx_j - C_j(x_j)$. The first-order condition is
$$
\frac{d\pi_j}{dp_j} = x_j + \frac{dx_j}{dp_j}p_j - \frac{dx_j}{dp_j}c_j = 0 \rightarrow p_j = c_j - \frac{x_j}{\frac{dx_j}{dp_j}}
$$
$$
p_j = c_j - \frac{s_j}{\frac{ds_j}{dp_j}}
$$
The FOC in the logit model is 
$$
p_j = c_j + \frac{1}{\alpha(1-s_j)}
$$
% See https://github.com/johnjosephhorton/berry1994/blob/master/berry.R for R code of this practice.

% See https://github.com/rkackerman/Industrial-Organization-HW-Example-One for MATLAB code of this pratice

In [179]:
# See the below link for Distribution
# https://juliastats.github.io/Distributions.jl/stable/starting/
using LinearAlgebra
using Random, Distributions
using Optim
using NLsolve
using DataFrames

In [180]:
# Parameters

J = 2 #Number of products
N = 100 #Number of individuals
M = 500 #Number of markets
Jchar = 1 #Number of product characteristics
iter = 100 #Number of iterations
β_0 = 5
β_x = 2
σ_d = 1 # or
# σ_d = 3
α = 1
γ_o = 1
γ_x = .5
γ_w = .25
σ_w = .25
σ_c = .25
df = DataFrame()

""
""


In [181]:
# Draw variables for Monte Carlo simulation    
d1 = Normal()
Random.seed!(400) #Setting the seed
Variables = rand(d1, 4, J)
X = Variables[1,:]
ξ = Variables[2,:]
w = Variables[3,:]
ω = Variables[4,:]
d2 = Gumbel()
Random.seed!(401) #Setting the seed
ϵ = rand(d2 , J+1 , N) # error term

3×100 Array{Float64,2}:
 -0.692732  -0.772131  -0.611837  …  -0.518236   0.184831  -0.778814
 -0.176541   0.189508   1.18723      -0.25022   -0.891603   0.839889
  3.77288    0.344294   2.23483      -0.476165   1.03364    1.09074 

In [182]:
#= This doesn't work. My guess is because the findmax function is not continuous, hence, not differentiable
function equilibriumPrice(price)
    # price is equilibrium variable
    # I used FOC for the logit model
    d2 = Gumbel()
    Random.seed!(123) #Setting the seed
    u = rand(d2 , J+1 , N) # error term
    u = u .+ [β_0 .+ β_x*X + σ_d*ξ - α*price'; 0] # utility (unobserved)
    u_max, ind = findmax(u, dims = 1)
    for n in 1:N
        for j in 1:J+1
            choice_mat[j,n] = (u_max[n] <= u[j,n])
        end
    end
    s = sum( choice_mat, dims = 2)/N
    c = exp.(γ_o .+ γ_x*X + + σ_c*ξ + γ_w*w + σ_w*ω) # marginal cost (observed)
    equ = price' - c - 1 ./ ( α*(1 .- s[1:2]) )
    return equ
    end
=#

In [183]:
# Data Generating Process dgp
# This function is used to generate the observed data
# Observed data: p (prices), X(characteristics), c(marginal cost), s(market share), w(input characteristics)
function dgp(i)
    # Variables
    d1 = Normal()
    Random.seed!(100 + i) #Setting the seed
    Variables = rand(d1, 4, J)
    X = Variables[1,:] 
    ξ = Variables[2,:]
    w = Variables[3,:]
    ω = Variables[4,:]
    Random.seed!(101 + i) #Setting the seed
    ϵ = rand(d2, J+1, N)
    # Find the equilibrium prices
    initial_p = [.1  .1]
    solution = nlsolve( equilibriumPrice, initial_p)
    p = solution.zero'
    u = [β_0 .+ β_x*X + σ_d*ξ - α*p; 0] .+ ϵ
    u_max = maximum(u, dims = 1)
    choice_mat = (u_max .<= u)
    s = sum( choice_mat, dims = 2)/N
    c = exp.(γ_o .+ γ_x*X + + σ_c*ξ + γ_w*w + σ_w*ω)
    return X, w, p, c, s
end

#= This function is used to find the equilibrium prices
function equilibriumPrice(price)
    # price is the equilibrium variable
    # I used FOC for the logit model
    u = [β_0 .+ β_x*X + σ_d*ξ - α*price'; 0]
    s = exp.(u)/ sum( exp.(u) ) # Assume that heterogeniety error term is T1EV
    c = exp.(γ_o .+ γ_x*X + σ_c*ξ + γ_w*w + σ_w*ω) # marginal cost (observed)
    equ = price' - c - 1 ./ ( α*(1 .- s[1:2]) )
    return equ
end
=#
function equilibriumPrice(price)
    # price is equilibrium variable
    # I used FOC for the logit model
    u = [β_0 .+ β_x*X + σ_d*ξ - α*price'; 0] .+ ϵ
    u_max = maximum(u, dims = 1)
    choice_mat = (u_max .<= u)
    s = sum( choice_mat, dims = 2)/N
    c = exp.(γ_o .+ γ_x*X + + σ_c*ξ + γ_w*w + σ_w*ω) # marginal cost (observed)
    equ = price' - c - 1 ./ ( α*(1 .- s[1:2]) )
    return equ
end

function OLS(Y , X)
    
    β = ( (X'*X)^(-1) ) * X'*Y
    return β
#    var_β = ...
end

OLS (generic function with 1 method)

In [184]:
    initial_p = [.1  .1]
    solution = nlsolve( equilibriumPrice, initial_p)
    p = solution.zero'
    u = [β_0 .+ β_x*X + σ_d*ξ - α*p; 0] .+ ϵ # mean utility
    u_max = maximum(u, dims = 1)
    choice_mat = (u_max .<= u)
    s = sum( choice_mat, dims = 2)/N
    c = exp.(γ_o .+ γ_x*X + + σ_c*ξ + γ_w*w + σ_w*ω)

2-element Array{Float64,1}:
 1.051422219774897 
 2.6853047917710176

First I regress $\delta_j = ln(s_j) - ln(s_0)$ on $x_j$ and $p_j$ without regard to the endogeneity of prices.

In [189]:
dgp(300)
#Y = log.(s[1:2]) .- log( s[3])
#X1 = [ones(J) X p ]
#OLS(Y , X1)

MethodError: MethodError: no method matching *(::Int64, ::Tuple{Array{Float64,1},Array{Float64,1},Adjoint{Float64,Array{Float64,2}},Array{Float64,1},Array{Float64,2}})
Closest candidates are:
  *(::Any, ::Any, !Matched::Any, !Matched::Any...) at operators.jl:529
  *(::T<:Union{Int128, Int16, Int32, Int64, Int8, UInt128, UInt16, UInt32, UInt64, UInt8}, !Matched::T<:Union{Int128, Int16, Int32, Int64, Int8, UInt128, UInt16, UInt32, UInt64, UInt8}) where T<:Union{Int128, Int16, Int32, Int64, Int8, UInt128, UInt16, UInt32, UInt64, UInt8} at int.jl:54
  *(::Union{Int16, Int32, Int64, Int8}, !Matched::BigInt) at gmp.jl:475
  ...

In [186]:
βhat = zeros(Jchar, iter)
for i in 1:iter
    
# Computing Share of Products
        X = dgp(i)
        δ = X*β
        d_2 = Gumbel() # or we can use d = Normal()
        Random.seed!(101 + i) #Setting the seed
        ϵ = rand(d_2, J , N)
        u = δ .+ ϵ
        u_max, ind = findmax(u, dims = 1)
        choice_mat = (u .>= u_max)
        s = sum( choice_mat, dims = 2)/N
    # Maximum Likelihood Estimation
        β_0 = ones(Jchar)
        optimum = optimize(loglikelihood, β_0)
        MLE = optimum.minimum
        βhat[ :, i] = optimum.minimizer
end
βhat

UndefVarError: UndefVarError: β not defined

In [187]:
println( mean( βhat , dims = 2 ) )
println( var( βhat , dims = 2) )

[0.0]
[0.0]
